In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

Data folder is C:\Users\lfranceschi\DATASETS
Experiment save directory is  C:\Users\lfranceschi\EXPERIMENTS


#### Checking online hyperparameter optimization for `ReverseHG`.

In [3]:
def _test(method):
    try: ss.close()
    except: pass
    tf.reset_default_graph()
    ss = tf.InteractiveSession()
    v1 = tf.Variable([[1., -1., 1.], [2., 10., 2.]])

    v2 = tf.Variable([-1., 1., 0.2])
    lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape(), scalar=True)

    cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)
#     cost = tf.reduce_sum(lmbd*v2**2)

    
    
    _et = far.get_hyperparameter('eta', 0.1)
    io_optim = far.MomentumOptimizer(_et, far.get_hyperparameter('mu', .5))
    # io_optim = far.GradientDescentOptimizer(_et)

    io_optim_dict = io_optim.minimize(cost) 
    oo = tf.reduce_mean(v1*v2)
#     oo = tf.reduce_mean(v2)
    optim_oo = tf.train.AdamOptimizer(.01)
    farho = far.HyperOptimizer(hypergradient=method())
    farho.minimize(oo, optim_oo, cost, io_optim)
    return ss, farho, cost, oo

#### REVERSE

In [4]:
ss, farho, cost, oo = _test(far.ReverseHg)

tf.global_variables_initializer().run()

# execution with gradient descent! (looks ol)
farho.run(10)

hypergrads_rev = ss.run(far.utils.hypergradients())
hypergrads_rev

[-0.018393531, 0.055180617, 0.0036787055, 0.53872281, -0.076542258]

In [5]:
ss, farho, cost, oo = _test(far.ReverseHg)

tf.global_variables_initializer().run()

# execution with gradient descent! (looks ol)
farho.run(10, online=True, _skip_hyper_ts=True)

hypergrads_online = ss.run(far.utils.hypergradients())
hypergrads_online

[-0.018393531, 0.055180617, 0.0036787055, 0.53872281, -0.076542258]

In [6]:
print(np.linalg.norm(np.array(hypergrads_online) - np.array(hypergrads_rev)))

0.0


#### OK... SOUNDS GOOD

In [7]:
ss.run(tf.trainable_variables())

[array([[ 0.51925904, -0.51925904,  0.51925904],
        [ 1.03851807,  5.19259167,  1.03851807]], dtype=float32),
 array([ 0.03254529, -0.03254529, -0.00650906], dtype=float32)]

In [8]:
'''with online optimization parameters are not reinitialize. It corresponds to performin
a warm restart for solving the inner optimization problem.
'''
farho.run(10, online=True, _skip_hyper_ts=True)

ss.run(tf.trainable_variables())

[array([[ 0.24525888, -0.24525888,  0.24525888],
        [ 0.49051777,  2.45258904,  0.49051777]], dtype=float32),
 array([ -2.79949629e-04,   2.79949629e-04,   5.59900072e-05], dtype=float32)]

In [9]:
# check that indeed the paramters with batch hyperparameter optimization are different...
farho.run(10, online=False,  _skip_hyper_ts=True)

ss.run(tf.trainable_variables())

[array([[ 0.51925904, -0.51925904,  0.51925904],
        [ 1.03851807,  5.19259167,  1.03851807]], dtype=float32),
 array([ 0.03254529, -0.03254529, -0.00650906], dtype=float32)]